In [1]:
import pandas as pd
import numpy as np

In [2]:
df_b = pd.read_csv('./result_670/bup_online_compare_0508_step7.csv')

In [3]:
df_b.shape

(4096, 15)

In [4]:
df_b.head()

,wid,dcid,model,fbd,fds,asales,fsales,vendibilitydays,cate3,salesband,dt,RDCKey,ProductKey,ypred_raw,salesForecast
0,6932125,10,intercept_lr_model_id,2018-05-08,7,2.0,2.156000,7,736,E,2018-05-08,10.0,6932125.0,1.529359,2.0
1,929931,9,lr_model_id,2018-05-08,7,0.0,0.000000,7,736,F,2018-05-08,9.0,929931.0,0.113980,0.0
2,929931,4,lr_model_id,2018-05-08,7,0.0,0.000000,7,736,F,2018-05-08,4.0,929931.0,0.190791,0.0
3,1195224,9,lr_model_id,2018-05-08,7,0.0,1.797853,7,736,F,2018-05-08,9.0,1195224.0,1.223879,0.0
4,1195224,316,lr_model_id,2018-05-08,7,2.0,2.509512,7,736,E,2018-05-08,316.0,1195224.0,1.139005,2.0


In [5]:
df_b.cate3.unique()

array([736])

In [5]:
df_l = pd.read_csv('./result_670/transfer_test_result_736_0508-step28.csv')

In [6]:
df_l.shape

(197176, 8)

In [7]:
df_l.head()

,Date,RDCKey,ProductKey,ypred,model_2_ypred,sales,salesForecast,ensemble
0,2018-05-08,3.0,127768,386.14040,291.925724,304.0,304.0,332.487686
1,2018-05-09,3.0,127768,386.14040,435.933684,217.0,217.0,409.529056
2,2018-05-10,3.0,127768,358.15082,354.450006,31.0,29.0,356.290807
3,2018-05-11,3.0,127768,148.93347,209.619713,7.0,7.0,174.140925
4,2018-05-12,3.0,127768,113.28675,104.959811,3.0,3.0,108.964430


In [8]:
df = df_l[df_l.Date <= '2018-05-14']

In [9]:
df.shape

(49294, 8)

In [10]:
df1 = df.groupby(['RDCKey','ProductKey']).sum().reset_index()

In [11]:
df1.shape

(7042, 7)

In [12]:
df1.head()

,RDCKey,ProductKey,ypred,model_2_ypred,sales,salesForecast,ensemble
0,3.0,127768,1896.243790,1956.718752,590.0,588.0,1910.310166
1,3.0,163890,9.545874,9.318132,20.0,20.0,9.384403
2,3.0,189018,2.500754,5.277123,3.0,3.0,3.320952
3,3.0,189019,1.109161,1.392299,1.0,1.0,1.225706
4,3.0,189028,8.564203,11.239456,4.0,4.0,9.630726


In [13]:
df2 = pd.merge(df_b, df1, how='left', on=['ProductKey','RDCKey','salesForecast'])

In [14]:
df2.shape

(4096, 19)

In [15]:
df2.head()

,wid,dcid,model,fbd,fds,asales,fsales,vendibilitydays,cate3,salesband,dt,RDCKey,ProductKey,ypred_raw,salesForecast,ypred,model_2_ypred,sales,ensemble
0,6932125,10,intercept_lr_model_id,2018-05-08,7,2.0,2.156000,7,736,E,2018-05-08,10.0,6.93212e+06,1.529359,2.0,1.630960,1.413495,2.0,1.479321
1,929931,9,lr_model_id,2018-05-08,7,0.0,0.000000,7,736,F,2018-05-08,9.0,929931,0.113980,0.0,0.126004,1.047922,0.0,0.223177
2,929931,4,lr_model_id,2018-05-08,7,0.0,0.000000,7,736,F,2018-05-08,4.0,929931,0.190791,0.0,0.229780,0.988850,0.0,0.367644
3,1195224,9,lr_model_id,2018-05-08,7,0.0,1.797853,7,736,F,2018-05-08,9.0,1.19522e+06,1.223879,0.0,0.908958,0.809936,0.0,0.821542
4,1195224,316,lr_model_id,2018-05-08,7,2.0,2.509512,7,736,E,2018-05-08,316.0,1.19522e+06,1.139005,2.0,1.127578,1.113272,2.0,1.002888


In [18]:
df2['final_ensemble'] = 2 / ( 1 / df2.ypred_raw + 1 / df2.model_2_ypred)

df2['final_ensemble_2'] = (df2.ypred_raw + df2.model_2_ypred) / 2

df2['final_ensemble_3'] = (0.303/(0.294+0.303)) * df2.ypred_raw + (0.294/(0.294+0.303)) * df2.model_2_ypred

df2.head()

print df2.final_ensemble.sum()
print np.sum(np.abs(df2.final_ensemble - df2.salesForecast)) / np.sum(df2.salesForecast)
#print np.sum(np.abs(df2.final_ensemble - df2.asales)) / np.sum(df2.asales)

print df2.final_ensemble_2.sum()
print np.sum(np.abs(df2.final_ensemble_2 - df2.salesForecast)) / np.sum(df2.salesForecast)
#print np.sum(np.abs(df2.final_ensemble_2 - df2.asales)) / np.sum(df2.asales)

print df2.final_ensemble_3.sum()
print np.sum(np.abs(df2.final_ensemble_3 - df2.salesForecast)) / np.sum(df2.salesForecast)
#print np.sum(np.abs(df2.final_ensemble_3 - df2.asales)) / np.sum(df2.asales)

#print df2.asales.sum()
print df2.salesForecast.sum()
print df2.fsales.sum()
print df2.ypred_raw.sum()
print df2.model_2_ypred.sum()
print df2.ensemble.sum()
print "\n"
print np.sum(np.abs(df2.fsales - df2.salesForecast)) / np.sum(df2.salesForecast)
print np.sum(np.abs(df2.ypred_raw - df2.salesForecast)) / np.sum(df2.salesForecast)
print np.sum(np.abs(df2.model_2_ypred - df2.salesForecast)) / np.sum(df2.salesForecast)
print np.sum(np.abs(df2.ensemble - df2.salesForecast)) / np.sum(df2.salesForecast)
print "\n"
#print np.sum(np.abs(df2.fsales - df2.asales)) / np.sum(df2.asales)
#print np.sum(np.abs(df2.ypred_raw - df2.asales)) / np.sum(df2.asales)
#print np.sum(np.abs(df2.model_2_ypred - df2.asales)) / np.sum(df2.asales)
#rint np.sum(np.abs(df2.ensemble - df2.asales)) / np.sum(df2.asales)



274082.83252601494
0.27091324921562615
283448.86789599096
0.26378087154976454
283116.68126170215
0.26374950484411336
293802.0
218843.0118420045
261413.82115483301
305483.91463714896
252068.19817928594


0.3743037158219498
0.2900279632137051
0.3014032072711533
0.32032265378501634


